<a href="https://colab.research.google.com/github/nunocesarsa/Examples/blob/main/ToClean_CroppingS2Data_ByGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to Google Earth Engine

In [ ]:
#importing Earth Engine package
import ee

#then we need to authenticate
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Fz-JkxhRqoYb7H-J_oNtkPJFn3Y8BX_TY0F6seMWIeU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0AGD5qA36AfjbEgfyOTqyWztQ8M7XQHGrCMa5ip5KXrubNPZH97GwK8

Successfully saved authorization token.


Mounting google drive to save the outputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Installing needed packages

In [ ]:
!pip install pyrsgis
!pip install rasterio
!pip install pyproj
!pip install geopandas

     |████████████████████████████████| 18.2MB 1.2MB/s 
     |████████████████████████████████| 10.9MB 2.7MB/s 
     |████████████████████████████████| 931kB 2.6MB/s 
     |████████████████████████████████| 14.7MB 313kB/s 


Importing packages that might (or not) be useful later

In [ ]:
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd

import os
from pyrsgis import raster

import rasterio
import rasterio.plot
import pyproj
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import geopandas as gpd


from pyrsgis.convert import changeDimension

Warning! matplotlib_scalebar library not found.


Exploring loaded shapefiles

In [ ]:
redouan_shp = gpd.read_file("/content/drive/My Drive/Redouan/Area_MapsWGS84_CutBlocks.shp")
redouan_shp.plot()

#and we can also plot each subsection individually
for i in redouan_shp['id']:
  area_sel = redouan_shp.loc[redouan_shp['id']==i]
  area_sel.plot()



In [ ]:
from datetime import datetime
from pytz import timezone    

#to have a sense of the time it takes
berlin_time = timezone('Europe/Berlin')


Core loop - Please load auxiliar functions at the bottom 


In [ ]:
#setting parameters for inversion
n_traits= 4
n_samples = 4000

label_names = ["cab","cw","cm","lai"]
training_noise_level = .05

number_epochs = 300

#output folder
path2outfld = '/content/drive/My Drive/Redouan'

#auxiliar function
def coords_from_gpd(poly):
  return eval(poly.to_json())['features'][0]['geometry']['coordinates']

for i in redouan_shp['id']:

  #selects the ith shape block and creates a ee.shape out of it
  area_sel = redouan_shp.loc[redouan_shp['id']==i]
  #area_sel.plot()
  area_ee = ee.Geometry.Polygon(coords_from_gpd(area_sel))


  #fetches all available iamges in the area based on the shape and time
  s2_collection = (ee.ImageCollection("COPERNICUS/S2_SR").
                   select(['B2','B3','B4',
                           'B5','B6','B7',
                           'B8A','B11','B12']).
                   filter(ee.Filter.date('2019-06-01','2019-07-31')).
                   filterBounds(area_ee).
                   filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)))
  count = s2_collection.size()
  print('Nr of images in collection: ', str(count.getInfo())+'\n')

  k_set = 1
  #here, it will download and retrieve the parameters using a multitask neural network
  for j in range(count.getInfo()):
    print('Downloading and retrieving parameters on area '+ str(j+1) + ' image ' + str(i+1)) + " of " + count.getInfo()


    #fetching the image
    mid_step = s2_collection.toList(count)
    s2_img = ee.Image(mid_step.get(j))
    s2_img_meta = s2_img.getInfo()

    #and printing out the name to confirm i got the right thing
    out_s2_name = s2_img_meta.get('properties',{}).get('PRODUCT_ID')[0:19]+"AreaID"+str(i) + "Set_"+ str(k_set)
    print("Loaded:" + s2_img_meta.get('properties',{}).get('PRODUCT_ID'))

    ################################################# the bulk data downloading starts here  #################################################

    #Creates the task for EE
    task_config = {
      'image': s2_img,
      'fileFormat': 'GeoTIFF',
      'folder': 'Redouan_out',
      'fileNamePrefix': out_s2_name,
      'description': "clipped area",
      'scale':20,
      'region':area_ee}
    
    #starts the task
    print("Google Earth engine task started")
    task = ee.batch.Export.image.toDrive(**task_config )
    task.start()

    #creates a loop that is infinite until the its finished
    flag = task.status()['state']
    while flag != 'COMPLETED':
      flag = task.status()['state']
      #print(k,flag)
      if flag == 'FAILED':

        print("GEE data cropping failed - proceeding to next image")
        print("Failed: "+ out_s2_name)
        flag = 'COMPLETED' 
    print("Saved at: " + "/content/drive/My Drive/Redouan_out/" + out_s2_name + ".tif")


     ################################################# the bulk model training starts here  #################################################

    #selecting the sensor noise convolution matrix
    sens_tag = out_s2_name[0:3]
    if sens_tag == 'S2A':
      filepath="/content/drive/My Drive/S2_Response.csv"
      print("Sensor found: S2A")
    if sens_tag == 'S2B':
      filepath="/content/drive/My Drive/S2_Responses_S2B.csv"
      print("Sensor found: S2B")

    #LAtin hypercube sampling on a space of [0-1]
    LHS_train = lhsmdu.createRandomStandardUniformMatrix(n_traits,n_samples)

    #creating traits table
    pd_trait = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
    pd_trait.columns = label_names
    pd_trait["cab"]=pd_trait["cab"]*max_cab+min_cab
    pd_trait["cw"] =pd_trait["cw"] *max_cw+min_cw
    pd_trait["cm"] =pd_trait["cm"] *max_cm+min_cw
    pd_trait["lai"]=pd_trait["lai"]*max_lai+min_lai    

    #numpy format
    np_trait = pd_trait.iloc[:,:].values

    #fetching the metadata info on the sensor position
    band_list = ['B2','B3','B4','B5','B6','B7','B8A','B11','B12']
    inc_azi = 0 #starting value
    inc_zen = 0
    for k in band_list:
      #print('MEAN_INCIDENCE_AZIMUTH_ANGLE_'+i)
      #print(s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_'+i))
      inc_azi = inc_azi + s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_'+k)/len(band_list)
      inc_zen = inc_zen + s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_'+k)/len(band_list)

    sol_azi = s2_img_meta.get('properties',{}).get('MEAN_SOLAR_AZIMUTH_ANGLE')
    sol_zen = s2_img_meta.get('properties',{}).get('MEAN_SOLAR_ZENITH_ANGLE')

    #and storing them on precise names as PROSAIL expects
    tts = sol_zen
    tto = inc_zen
    phi = inc_azi-sol_azi  #according to the details of prosail
    if phi < 0:
      print("Negative phi:" +str(phi) + "inverting substraction")
      phi = sol_azi - inc_azi

    print("Geometric parameters - tts: "+ str(tts) +" tto: "+ str(tto) + " phi: " + str(phi))


    #now we generate spectra using prosail 
    #the metadata of the input image MUST contribute here
    print("Generating the spectra")
    np_spect = Gen_spectra_data(pd_trait,tts,tto,phi)[:,[1,2,3,4,5,6,8,11,12]]

    #adding noise to the signal 
    print("Adding the noise of ",training_noise_level*100,"% to training data")
    np_spect_comb = np.apply_along_axis(combined_noise,1,np_spect,sigma=training_noise_level) # 5% noise on the data, combined

    #this probably would not be necessary but just in case i want to check the accuracy (basically sets aside 10% of the data)
    index = list(range(len(np_spect)))
    index10 = rdm.sample(index,math.ceil(len(index)*.1)) #randomly selects 10% of te data (aproximately)
    index90 = [x for x in index if x not in index10] #makes a list with the remaining

    vl_spect_df_comb = np_spect_comb[index10,]
    tr_spect_df_comb = np_spect_comb[index90,]

    vl_trait_df = np_trait[index10,]
    tr_trait_df = np_trait[index90,]

    #this is a function to re-scale inputs for the neural networks
    scaler = MinMaxScaler()   

    #spectral data:
    X_train = tr_spect_df_comb
    #Trait data:
    Y_train = tr_trait_df
    
    #print("Training the gaussian processes @" + datetime.now(berlin_time).strftime('%Y-%m-%d %H:%M:%S'))
    #gpr_ml_comb.fit(X_train,Y_train)
    #print("Finished @" + datetime.now(berlin_time).strftime('%Y-%m-%d %H:%M:%S'))

    #preparing data for the multi-task network
    scaler.fit(Y_train)
    Y_train_norm = scaler.transform(Y_train)

    cab_train = Y_train_norm[:,0]
    cw_train = Y_train_norm[:,1]
    cm_train = Y_train_norm[:,2]
    lai_train = Y_train_norm[:,3]

    print("Training the multi task neural network @ " + datetime.now(berlin_time).strftime('%Y-%m-%d %H:%M:%S'))
    model_comb.fit(X_train,[cab_train,cw_train,cm_train,lai_train],epochs=number_epochs,verbose=0)#hides spam on the output
    print("Finished @ " + datetime.now(berlin_time).strftime('%Y-%m-%d %H:%M:%S'))
    #here i will put a flow to print the predictions on the out-of-bag samples later

    #Fetching the data for the prediction
    print("Fetched image: "+'/content/drive/My Drive/Redouan_out/'+ out_s2_name + ".tif")

    #reads the raster
    ds, bands = raster.read('/content/drive/My Drive/Redouan_out/'+ out_s2_name + ".tif")
    #ds holds the information to reconstruct the raster
    #bands is the actual information
    
    #creates an array of  n(number of pixels) by 9 (bands of sentinel 2) -> also divides by 10000 to transform the bits to reflectance
    bandByPixel = convert_rst(bands)

    #Gaussian processes easily runs out of memory when doing predictions so we have to "reduce the number of cells"
    #from sklearn.model_selection import KFold
    #kf = KFold(n_splits=50)
    #kf.split(bandByPixel)

    ############## predicting the neural network, hopefully i don't run out of memory on this procedure
    mtn_pred = model_comb.predict(bandByPixel)

    #the output is a n(number of pixels) by 4 bands but the values have to be "scaled back"
    cab_mtn = mtn_pred[0]
    cw_mtn  = mtn_pred[1]
    cm_mtn  = mtn_pred[2]
    lai_mtn = mtn_pred[3]

    np_Y_ann_pred = np.array(cab_mtn)
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,cw_mtn))
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,cm_mtn))
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,lai_mtn))
    #inverting the min-max scaling

    np_Y_ann_pred = scaler.inverse_transform(np_Y_ann_pred)

    #and now we reconstruct the raster
    mtn_cab_rst = np.reshape(np_Y_ann_pred[:,0],(ds.RasterYSize,ds.RasterXSize))
    mtn_cw_rst = np.reshape(np_Y_ann_pred[:,1],(ds.RasterYSize,ds.RasterXSize))
    mtn_cm_rst = np.reshape(np_Y_ann_pred[:,2],(ds.RasterYSize,ds.RasterXSize))
    mtn_lai_rst = np.reshape(np_Y_ann_pred[:,3],(ds.RasterYSize,ds.RasterXSize))

    #creating the filename for output
    mtn_cab_name = '/content/drive/My Drive/Redouan_out/' + out_s2_name + "_MTN_cab_05"+".tif"
    mtn_cw_name =  '/content/drive/My Drive/Redouan_out/' + out_s2_name + "_MTN_cw_05"+".tif"
    mtn_cm_name =  '/content/drive/My Drive/Redouan_out/' + out_s2_name + "_MTN_cm_05"+".tif"
    mtn_lai_name = '/content/drive/My Drive/Redouan_out/' + out_s2_name + "_MTN_lai_05"+".tif"

    #saving the rasters
    print("Saving the biophysical paramaters retrievals")
    raster.export(mtn_cab_rst, ds, mtn_cab_name, dtype='float')
    raster.export(mtn_cw_rst, ds, mtn_cw_name, dtype='float')
    raster.export(mtn_cm_rst, ds, mtn_cm_name, dtype='float')
    raster.export(mtn_lai_rst, ds, mtn_lai_name, dtype='float')

    k_set = k_set+1



Auxiliar processing functions 

In [ ]:
#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
#lets try to do a LHS
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
!pip install pysptools

In [ ]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
#importing stuff 
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

#the beutiful R like data frame
import pandas as pd

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#a few more stuff for random
import random as rdm
import math



#package to for operations on spectral data
import pysptools as sptool 
from pysptools import distance


from sklearn.model_selection import KFold # import KFold

#machine learning stuff
#NEURAL NETWORK - Keras will be updated soon so this colab will also have to be changed

from sklearn.neural_network import MLPRegressor as ANN_reg #this is a simpler neural network package
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.layers import GaussianNoise

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#ignore the warning for now

#Random FOREST
# Fitting Random Forest Regression to the dataset 
# import the regressor 
from sklearn.ensemble import RandomForestRegressor 

#Gaussian processes
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel, ExpSineSquared, RationalQuadratic, RBF

#aux functions
from sklearn.preprocessing import MinMaxScaler #this is to standardize the input data [not used for now]
from sklearn import metrics

#for model storing -sklearn
import pickle

#for model storing keras
from keras.models import model_from_json

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#curstom function to re-organize the raster into a structure i can use
def convert_rst(bands):
  bandByPixel = changeDimension(bands)/10000. #we have to devide all values by 10k - its a conversion from bits to reflectances
  #bandByPixel_t = np.transpose(bandByPixel)
  return bandByPixel


#custom function to control prosail inputs
def custom_prosail(cab,cw,cm,lai,tts,tto,phi):
  import prosail
  #default parameters
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters - got this from R package https://www.rdocumentation.org/packages/hsdar/versions/0.4.1/topics/PROSAIL
  #sun and viewing angle
  #tts=30. #observation and solar position parameters
  #tto=10. 
  #psi=0.
  #for now i put them by hand but they should be an input of a custom function
  tts=tts #solar zenith angle
  tto=tto #sensor zenith angle
  psi=phi
  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)

def Prosail2S2(path2csv,spectra_input):
  #importing pandas
  import pandas as pd
  import numpy
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  
  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column

#please LOAD THTE FILE NOW
#filepath="/content/drive/My Drive/S2_Response.csv"
#filepath="/content/S2_Responses_S2B.csv"
#filepath="/content/drive/My Drive/S2_Response.csv"


def Gen_spectra_data(traits,tts,tto,phi):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  for i in range(len(traits)):
    #n_t = pd_train_traits["n"][i]
    cab_t = traits["cab"][i]
    #car_t = pd_train_traits["car"][i]
    #cbrown_t = pd_train_traits["cbrown"][i]
    cw_t = traits["cw"][i]
    cm_t = traits["cm"][i]
    lai_t = traits["lai"][i]

    if k == 1:
      tr_rho_s = custom_prosail(cab_t,cw_t,cm_t,lai_t,tts,tto,phi)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)
      #plt.plot ( x, tr_rho_s, ':', label="Training prosail")
      #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,cw_t,cm_t,lai_t,tts,tto,phi)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))
      #plt.plot ( x, tr_rho_t, ':')

    k = k+1


  rho_samples=tr_rho_s


  return rho_samples




#Function to store the outputs into a table
column_names=["Model",
              "NoiseLevel",
              "NoiseType",
              "OutOfBag",
              "KFold_tr_samples",
              "KFold_vl_samples",
              "Variable",
              "Fold_nr",
              "ExplVar",
              "Max_err",
              "Mean_abs_Err",
              "Mean_sqr_err",
              #"Mean_sqr_lg_err",
              "Median_abs_err",
              "r2",
              "MAPE",
              "ModelName"]
              #"Mean_poiss_dev",
              #"Mean_gamma_dev"]
              #"Mean_tweed_dev"]

#mape is not existant in the package so we have to create it:
#https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
#from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  #Here, the file is used for saving
  #creating a df to receive the data


def calc_metrics(MDL,NoiseLevel,NoiseType,oob_samples,kf_tr,kf_vl,Variable,Fold,Ref,Pred,Modelname):

  out_list = {"Model":MDL,
              "NoiseLevel":NoiseLevel,
              "NoiseType":NoiseType,
              "OutOfBag":oob_samples,
              "KFold_tr_samples":kf_tr,
              "KFold_vl_samples":kf_vl,
              "Variable":Variable,
              "Fold_nr":Fold,
              "ExplVar": metrics.explained_variance_score(Ref, Pred),
              "Max_err": metrics.max_error(Ref, Pred),
              "Mean_abs_Err": metrics.mean_absolute_error(Ref, Pred),
              "Mean_sqr_err": metrics.mean_squared_error(Ref, Pred),
              "Median_abs_err" : metrics.median_absolute_error(Ref, Pred),
              "r2": metrics.r2_score(Ref, Pred),
              "MAPE": mean_absolute_percentage_error(Ref, Pred),
              "ModelName":Modelname}


  return out_list

In [ ]:
def combined_noise(ref,sigma):

  #ref is an input vector and sigma is the standar deviation of the noise

  #this format implies some values go negative or above 1 which is terrible

  #the multiplier parameter is solid
  #mp_noise = ref*(1 + np.random.normal(0,scale=2*sigma,size=ref.shape[0]))

  #this form makes the value go over 1 and under 0 from time to time
  #ad_noise = 0#+np.random.normal(0,sigma)

  #truncated version - the limits are on 2*sigma to give it a leeway 
  #ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])

  #the only way i can think to force it doesn't go over 1 or under 0, is to use a naive verifier
  
  #sigma is any value between 0 and 1, reflecting "% percent error"
  #notice i give leeway to sigma by doubling the interval but the truncated version should be stronger againt bad values
  
  mp_noise = ref*(1 + stats.truncnorm.rvs(0-sigma*4,0+sigma*4,loc=0,scale=sigma*2,size=ref.shape[0]))
  ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])
  out_ref = mp_noise+ ad_noise


  #this looks fun but breaks the speed of processing....
  # while max(out_ref) > 1 or min(out_ref)<0:
  #   #print("i am in the loop!")
  #   mp_noise = ref*(1 + stats.truncnorm.rvs(0-sigma*4,0+sigma*4,loc=0,scale=sigma*2,size=ref.shape[0]))
  #   ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])
  #   out_ref = mp_noise+ ad_noise
  #   #print(out_ref)

  #making everything that goes under, be 0 or 1
  out_ref[out_ref>1]=1
  out_ref[out_ref<0]=0

  return out_ref

def inverted_noise(ref,sigma):

  #ref is an input vector and sigma is the standar deviation of the noise


  #in this case, both of these are producing improper results - less than 0 or more than 1 oftne. 
  #mp_noise = 1-(abs(1-ref)*(1 + np.random.normal(0,scale=2*sigma,size=ref.shape[0])))
  #ad_noise = 0#+np.random.normal(0,sigma)

  mp_noise = 1-(1-ref)*(1 + stats.truncnorm.rvs(0-sigma*4,0+sigma*4,loc=0,scale=sigma*2,size=ref.shape[0]))
  ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])

  out_ref = mp_noise+ ad_noise

  #this looks fun but breaks the speed of processing...
  # while max(out_ref) > 1 or min(out_ref)<0:
  #   #print("i am in the loop!")
  #   mp_noise = 1-(1-ref)*(1 + stats.truncnorm.rvs(0-sigma*4,0+sigma*4,loc=0,scale=sigma*2,size=ref.shape[0]))
  #   ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])
  #   out_ref = mp_noise+ ad_noise
  #   #print(out_ref)  while 


  #making everything that goes under, be 0 or 1
  out_ref[out_ref>1]=1
  out_ref[out_ref<0]=0

  return out_ref



In [ ]:
pr_ml_comb = GaussianProcessRegressor(n_restarts_optimizer=25, # -> 1*RBF(1.0) is the default kernel
                                       normalize_y=True,
                                       random_state=0,
                                       alpha=.05) #this is for noise

gpr_ml_inve = GaussianProcessRegressor(n_restarts_optimizer=25, # -> 1*RBF(1.0) is the default kernel
                                       normalize_y=True,
                                       random_state=0)

#multi-task neural network - 16 x 8 shared x 4 individual
inputs = Input(shape=(9,))
#noise = GaussianNoise(.05)(inputs) #im not sure this is how we use this function
sub1 = Dense(16, activation='tanh')(inputs)
sub2 = Dense(8, activation='tanh')(sub1)
cab1 = Dense(4, activation='sigmoid')(sub2)
cw1  = Dense(4, activation='sigmoid')(sub2)
cm1  = Dense(4, activation='sigmoid')(sub2)
lai1 = Dense(4, activation='sigmoid')(sub2)
cab2 = Dense(1, activation='linear')(cab1)
cw2  = Dense(1, activation='linear')(cw1)
cm2  = Dense(1, activation='linear')(cm1)
lai2 = Dense(1, activation='linear')(lai1)


#building the model using keras api
model_comb = Model(inputs=inputs, outputs=[cab2,cw2,cm2,lai2])
model_inve = Model(inputs=inputs, outputs=[cab2,cw2,cm2,lai2])

model_comb.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_inve.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

#metas:
number_epochs = 300

In [ ]:
n_traits=4 #I will test on 4 varying traits: cab, car, cw,cm,lai

#generating a LHS hypercube (it uses a 0 to 1 interval that can be used as a multiplier against the different traits)
np.random.seed(0)


#the values here are 1 less than the max so i can add a value later to it
#max_n=1 
max_cab=121. #add 1
#max_car=44. #add 1
#max_cbrown= 9.99 #add 0.01
max_cw=0.008 #add 0.001 
max_cm=0.008 #0.001
max_lai = 9.9 #add 0.1

min_cab = 10.
min_cw = 0.002
min_cm = 0.002
min_lai = .5

#labelling
#label_names = ["cab","cw","cm","lai"]